# Python for Finance: Plotting stock market data

<b> YouTube Tutorial </b> (Published: Apr 21, 2021): https://youtu.be/yQtT_4RgT2k

In this tutorial we follow on from importing financial data to then graphing financial data in timeseries chart format with plotly candlestick charts.
    https://plotly.com/python/candlestick-charts/

#### Step 1: import the dependencies
Ensure that plotly is installed, if not follow guidelines in last video Python for Finance: Get stock market data.

In [ ]:
## This is required for pandas_datareader on google colab - then you need to restart runtime
!pip install --upgrade pandas_datareader

     |████████████████████████████████| 109 kB 5.4 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)

pd.options.plotting.backend = 'plotly'

#### Step 2: get stock market data
Choose a date range and select stock to chart.

In [ ]:
end = dt.datetime.now()
start = dt.datetime(2015,1,1)

df = pdr.get_data_yahoo('CBA.AX', start, end)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-01,85.277962,84.661308,84.959686,85.277962,949439.0,60.323940
2015-01-04,85.775269,85.049202,85.238182,85.486832,1351651.0,60.471676
2015-01-05,85.337639,84.412651,84.641411,84.840332,2477275.0,60.014389
2015-01-06,85.029312,84.094376,84.850281,84.651360,2127190.0,59.880692
2015-01-07,85.188446,84.671249,85.079041,84.929848,1997761.0,60.077686


#### Step 3: Create Moving Average terms
For this we use pandas .rolling function and specify the rolling window parameter.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
By choosing a rolling window of 50/200, inherently this requires 50/200 rows of data before it can compute the mean. We end up with NaN (Not a Number), we can resolve this (if we would like to have a continuous MA line throughout this period) by specifying the min_periods parameter.

In [ ]:
df['MA50'] = df['Close'].rolling(window=50, min_periods=0).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=0).mean()
df['MA200'].head(30)

Date
2015-01-01    85.277962
2015-01-04    85.382397
2015-01-05    85.201708
2015-01-06    85.064121
2015-01-07    85.037267
2015-01-08    85.127113
2015-01-11    85.135875
2015-01-12    85.098932
2015-01-13    84.891169
2015-01-14    84.708051
2015-01-15    84.563651
2015-01-18    84.404363
2015-01-19    84.300184
2015-01-20    84.294009
2015-01-21    84.325789
2015-01-22    84.415760
2015-01-26    84.577055
2015-01-27    84.724296
2015-01-28    84.911528
2015-01-29    85.108382
2015-02-01    85.303065
2015-02-02    85.512601
2015-02-03    85.720781
2015-02-04    86.014390
2015-02-05    86.272972
2015-02-08    86.509751
2015-02-09    86.716466
2015-02-10    86.882484
2015-02-11    87.013731
2015-02-12    87.201541
Name: MA200, dtype: float64

#### Step 4: Create plotly fig / subplot

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
               vertical_spacing=0.10, subplot_titles=('CBA', 'Volume'),
               row_width=[0.2, 0.7])

fig.add_trace(go.Candlestick(x=df.index, open=df["Open"], high=df["High"],
                low=df["Low"], close=df["Close"], name="OHLC"),
                row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df["MA50"], marker_color='grey',name="MA50"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["MA200"], marker_color='lightgrey',name="MA200"), row=1, col=1)

fig.add_trace(go.Bar(x=df.index, y=df['Volume'], marker_color='red', showlegend=False), row=2, col=1)

fig.update_layout(
    title='CBA historical price chart',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Price ($/share)',
        titlefont_size=14,
        tickfont_size=12,
    ),
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    paper_bgcolor='LightSteelBlue'
)

fig.update(layout_xaxis_rangeslider_visible=True)
fig.show(renderer="colab")

